In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import codecs

# UD_Dutch_LassySmall
data_dir = '/home/jvdzwaan/data/UD_Dutch-LassySmall/'
test = data_dir + 'nl_lassysmall-ud-test.conllu'
train = data_dir + 'nl_lassysmall-ud-train.conllu'

Spacy uses a different numbering of the heads than conllx for Dutch. To transform conll x to spacy format, the root word (with head 0) should be the index in the word list. And all other words should be decremented with 1.

In [3]:
def get_sentence(data):
    words = []
    heads = []
    deps = []
    parts = data.split('\n')
    if parts:
        for part in parts:
            if not part.startswith('#'):
                d = part.split()
                if len(d) > 6:
                    words.append(d[1])
                    heads.append(int(d[6]))
                    deps.append(d[7])
    new_heads = []
    for indx, h in enumerate(heads):
        if h == 0:
            new_heads.append(indx)
        else:
            new_heads.append(h-1)
        
    return words, new_heads, deps
    

def read_connl(filepath):
    with codecs.open(filepath, encoding='utf-8') as f:
        data = f.read()
        
    sentences = data.split('\n\n')
    for sentence in sentences:
        yield get_sentence(sentence)

In [4]:
import sys
import random
data = []
for w, n, l in read_connl(train):
    data.append((w, n, l))
print len(data)
r = random.randint(0, len(data))
print r
print data[2606]

6642
110
([u'In', u'werkelijkheid', u'werd', u'hij', u'gevangen', u'genomen', u'door', u'de', u'Britse', u'generaal', u'Halkett', u'.'], [1, 5, 5, 5, 5, 5, 9, 9, 9, 5, 9, 5], [u'case', u'nmod', u'auxpass', u'nsubj', u'compound', u'root', u'case', u'det', u'amod', u'nmod', u'appos', u'punct'])


In [5]:
# source: examples/training/train_parser.py
import random

import spacy
from spacy.pipeline import DependencyParser
from spacy.gold import GoldParse
from spacy.tokens import Doc

def train_parser(nlp, train_data, left_labels, right_labels):
    labels = set(left_labels+right_labels)
    parser = DependencyParser(
                nlp.vocab,
                labels=labels)
    for itn in range(1000):
        random.shuffle(train_data)
        loss = 0
        for words, heads, deps in train_data:
            print words
            print heads
            print deps
            doc = Doc(nlp.vocab, words=words)
            gold = GoldParse(doc, heads=heads, deps=deps)
            #print gold.cand_to_gold
            #print gold.gold_to_cand
            #print gold.orig_annot
            print gold.is_projective
            loss += parser.update(doc, gold)
    parser.model.end_training()
    return parser

nlp = spacy.load('nl', tagger=False, parser=False, entity=False, add_vectors=False)

left_labels = set()
right_labels = set()
for _, heads, deps in data:
    for i, (head, dep) in enumerate(zip(heads, deps)):
        if i < head:
            left_labels.add(dep)
        elif i > head:
            right_labels.add(dep)
#print sorted(left_labels)
#print sorted(right_labels)
parser = train_parser(nlp, data, sorted(left_labels), sorted(right_labels))

[u'Hij', u'is', u'65', u'jaar', u'oud', u'geworden', u'.']
[4, 4, 3, 4, 4, 4, 4]
[u'nsubj', u'aux', u'nummod', u'nmod', u'root', u'cop', u'punct']
True


ValueError: Could not find a gold-standard action to supervise the dependency parser.
The GoldParse was projective.